In [2]:
from scipy.interpolate import interp1d
from scipy.integrate import odeint
from scipy.optimize import root
import matplotlib.pyplot as plt

In [3]:
import math

G  = 6.6730831e-8
c  = 2.99792458e10
MeV_fm3_to_pa = 1.6021766e35
c_km = 2.99792458e5 # km/s
mN = 1.67e-24 # g
mev_to_ergs = 1.602176565e-6
fm_to_cm = 1.0e-13
ergs_to_mev = 1.0/mev_to_ergs
cm_to_fm = 1.0/fm_to_cm
Msun = 1.988435e33
MeV_fm3_to_pa_cgs = 1.6021766e33
km_to_mSun = G/c**2

hbarc3 = 197.32700288295746**3

nucleon_mass = 938.04

pi = math.pi

In [4]:
def solve_tov(c_dens, rmax=30e5, rtol=1.0e-5, dmrel=10e-12, dr=100):
    c_dens *= MeV_fm3_to_pa_cgs / c ** 2

#     self.check_density(c_dens)

    r = np.arange(dr, rmax + dr, dr)

    P = self.press(c_dens)
    eden = self.en_dens(P)
    m = 4.0 * pi * r[0] ** 3 * eden

    psol = odeint(self.tov_eq, [P, m], r, rtol=rtol)
    p_R, m_R = psol[:,0], psol[:,1]

    # find the boundary of the star by finding point
    # where the mass stops to increase

    diff = (m_R[1:] - m_R[:-1])/m_R[1:]
    ind = -1
    for i, dm in enumerate(diff):
      if dm < dmrel and m_R[i] != 0:
        ind = i
        break

    M = m_R[ind - 1]
    R = r[ind - 1]

    r   = r[:ind]
    p_R = p_R[:ind]
    m_R = m_R[:ind]
    
    e_R = self.en_dens(p_R)
    
    return R / 1e5, M / Msun, (r, e_R, p_R, m_R)